In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("otp-ffb").getOrCreate();

In [3]:
print(spark)

In [4]:
df = spark.read.csv('/home/virtual/Downloads/Bovespa.csv', inferSchema=True, header=True)

In [5]:
# primeira otimizacao: criar um dataframe somente com as colunas necessarias
df.columns

['TypeReg',
 'Date',
 'BDICode',
 'Codneg',
 'MarketType',
 'Company',
 'Spec',
 'Prazot',
 'Currency',
 'Open',
 'Max',
 'Min',
 'Med',
 'Close',
 'Preofc',
 'Preofv',
 'Totneg',
 'Quatot']

In [ ]:
dfGroupBy = df.select([df.Company, df.Min, df.Max, df.Med])

In [ ]:
dfGroupBy.groupBy(dfGroupBy.Company).sum().show(10)

In [ ]:
dfGroupBy.rdd.getNumPartitions()

In [ ]:
df.groupBy(df.Company).sum().show()

In [ ]:
sc

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
dfRep = df.repartition(100)

In [ ]:
dfRep.rdd.getNumPartitions()

In [ ]:
dfRep.groupBy(dfRep.Company).sum().show(10)

In [ ]:
#otimizacao: persistir um dataframe que esta sendo muito utilizado

df.persist()

In [ ]:
df.unpersist()

In [ ]:
# otimizacao: salvar dados em format parquet
df.write.mode("overwrite").parquet("/home/virtual/Downloads/Bovespa.parquet")


In [ ]:
dfParquet = spark.read.parquet("/home/virtual/Downloads/Bovespa.parquet")

In [ ]:
dfParquet.show(5)

In [ ]:
dfGroupBy.groupBy(dfGroupBy.Company).sum().show(10)

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
df.createOrReplaceTempView("bovespa")

In [ ]:
sql = spark.sql("select `Company`, sum(`Min`), sum(`Max`), sum(`Med`) from bovespa group by `Company`")

In [ ]:
sql.show(10)

In [11]:
# UDF : User Definied Function

def upper_case(record):
    if record is None:
        return ""
    elif (len(record) > 5):
        return record.upper()
    else:
        return record.lower()
    


In [12]:
from pyspark.sql.functions import udf

In [13]:
upperCaseUDF = udf( lambda x: upper_case(x) )

In [14]:
dfUpperLower = df.withColumn( "UpperLower", upperCaseUDF(df.Company))

In [15]:
dfUpperLower.select(dfUpperLower.UpperLower).show(10)

+----------+
|UpperLower|
+----------+
|   ACESITA|
|   ACESITA|
|   ACESITA|
|   ACESITA|
|   ALBARUS|
|   ACESITA|
|   ACESITA|
|   ACESITA|
|ALPARGATAS|
|   ACESITA|
+----------+
only showing top 10 rows

